# I-LOFAR BST Tutorial

This tutorial is based on that of Pearse Murphy which can be found [here](https://www.dropbox.com/s/cgcn1nbqylzq8fe/KAIRA-UIT-DAT-506.zip?dl=0) and [here](https://github.com/murphp30/I_LOFAR_workshop_2020) for further reference. 

The following tutorial is aimed at familiarising the users with beamlet statistics (BST) data produced by the internation LOFAR station like the Irish Lofarstaion or I-LOFAR. BST data is the average power per beamlet that an antenna (or set of antennas) receives, integrated over a specified time period.

## Exercise 1: Reading in data

The first step in analysing any unfamiliar data is to read it into memory and find out some information about its structure.

Complete/answer the following:
* Read BST file it into memory. (Hint, `numpy` makes things easy)
* How many data points are there?
* How does this compare to the file size?
* How many subbands were observed? (Hint, need to know bitmode of observation)
* How long was the observation? 

In [ ]:
import os
import sys
import glob

from datetime import datetime, timedelta

import numpy as np

from matplotlib.colors import LogNorm
from matplotlib import dates
from matplotlib import pyplot as plt

In [ ]:
bstfile = "./BST_data/modea/20190612_133144_bst_00X.dat"
data = np.fromfile(bstfile)
print("Number of data points:",data.shape[0])
print("File size:",os.path.getsize(bstfile))
print("Bitmode:",os.path.getsize(bstfile)/data.shape[0])

#bitmode is 8 therefore 488 subbands (not always though)
t_len = data.shape[0]/488
print("Time samples:",t_len )

## Exercise 2:  Reshaping data

BST data is 2 dimensional, time and frequency. Using what we learned above reshape the data so that it is a 2D array with time on one axis and frequency on another.

In [ ]:
#data = data.reshape(int(t_len),488)
data = data.reshape(-1,488) 
print(data.shape)

## Exercise 3: Quick plot

We now have everything we need to make a quick plot of our data.

* Use `matplotlib` to produce a dynamic spectrum. Make sure to have time on the x-axis.
* What "mode" was this observation made in? (Hint, log your data or use log scale)
* Play around with vmin and vmax.

In [ ]:
#plt.imshow(data.T, aspect="auto", norm=LogNorm())
plt.imshow(np.log10(data.T), aspect="auto")
plt.figure()
plt.plot(np.log10(np.sum(data,0)))

## Exercise 4: Updating axes

Let's convert subband number to frequency and include the actual time for our axes.

Remember the conversion from subband number to frequency from earlier

$$\nu = (n-1+ \frac{s}{512})\frac{\nu_{clock}}{2},$$

where $n$ is the nyquist zone, $s$ is the subband number and $\nu_{clock}$ is the clock frequency.

In this example $n=1$ and $\nu_{clock} = 200$ MHz.

* Look at `solar_mode3_20190612_133144.sh` to find which subbands were used.
* Write a function to convert subband number to frequency.
* Using the timestamp in the BST file name, create an array of datetime objects for the time array.
* Replot the data with the correct frequency and time axis.

In [ ]:
def sb_to_freq(sb, zone=1):
    clock = 200 #MHz
    freq = (zone-1+sb/512)*(clock/2)
    return freq #MHz

In [ ]:
sbs = np.arange(7,495)
freqs = sb_to_freq(sbs)
# print(sbs)
# print(freqs)
obs_start = bstfile[len(bstfile)-27:len(bstfile)-12]

obs_start = datetime.strptime(obs_start, "%Y%m%d_%H%M%S")
print(obs_start)
obs_len  = timedelta(seconds = data.shape[0])
obs_end = obs_start + obs_len
t_lims = [obs_start, obs_end]

t_lims = dates.date2num(t_lims)
print(t_lims)
#you only really need start and end time for imshow but we'll do a full array anyway
t_arr = np.arange(0,t_len)
t_arr = t_arr*timedelta(seconds=1)
t_arr = obs_start+t_arr
t_arr = dates.date2num(t_arr)

In [ ]:
fig, ax = plt.subplots()
ax.imshow(data.T, aspect="auto", extent=[t_arr[0], t_arr[-1], freqs[-1], freqs[0]],
         vmin = np.percentile(data.T, 1), vmax = np.percentile(data.T, 99))
ax.xaxis_date()
plt.xlabel("Time")
plt.ylabel("Frequency (MHz)")

## Exercise 5: Remove background

How would you remove the effect of the antenna bandpass filter?

In [ ]:
data = data/np.mean(data[:100], axis=0)
fig, ax = plt.subplots()
ax.imshow(data.T, aspect="auto", extent=[t_arr[0], t_arr[-1], freqs[-1], freqs[0]],
         vmin = np.percentile(data.T, 1), vmax = np.percentile(data.T, 99))
ax.xaxis_date()
plt.xlabel("Time")
plt.ylabel("Frequency (MHz)")

## Exercise 5: Mode 357

Mode 357 is a common solar observation mode where the 488 beamlets are spread across Mode, 3, 5, and 7 to obtain wide freqency coverage of the sun.

| Mode | Beamlet |
| --- | --- |
| 3 | 0 - 199 |
| 5 | 200 - 399 |
| 7 | 400- 487 |

In [ ]:
bst_file = '20170908_083815_bst_00X.dat'
mode357_raw = np.fromfile('./BST_data/mode357/'+bst_file)
print(mode357_raw.shape, mode357_raw.dtype)

In [ ]:
n_times = mode357_raw.shape[0]/(488)
n_times

As before we need to reshape the data as this was an 8bit observatin we know there are 488 beamlets

In [ ]:
mode357 = mode357_raw.reshape(-1, 488)
mode357.shape

Now we can plot the full mode 357 data or dynamic spectrum

In [ ]:
plt.imshow(mode357.T, aspect='auto', norm=LogNorm(), origin='lower')

So we can see the data looks as we expect but it's not very useful withput the time and freqency information

Mode 357 is configured with the following subbands:
* 3, Zone=1: 54, 56, 58, ..., 454
* 5, Zone=2: 54, 56, 58, ..., 454
* 7: Zone=3, 54, 56, 58, ..., 230

Using the functions we defined above find the frequencies for these 

In [ ]:
freq_m3 = sb_to_freq(np.arange(54,454,2))
freq_m5 = sb_to_freq(np.arange(54,454,2), zone=2)
freq_m7 = sb_to_freq(np.arange(54,230,2), zone=3)
freq_m3.shape, freq_m5.shape, freq_m7.shape

As before we can extract the time information from the filename

In [ ]:
obs_start = bstfile[len(bstfile)-27:len(bstfile)-12]

obs_start = datetime.strptime(obs_start, "%Y%m%d_%H%M%S")
print(obs_start)
obs_len  = timedelta(seconds = data.shape[0])
obs_end = obs_start + obs_len
t_lims = [obs_start, obs_end]

t_lims = dates.date2num(t_lims)

We can extract the data for each of the modes

In [ ]:
m3 = mode357[:,0:200]
m5 = mode357[:,200:400]
m7 = mode357[:,400:]

m3.shape, m5.shape, m7.shape

Finally plot the mode 357 with the correct axis by combing what was done above for the other BST file.

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(nrows=3, ncols=1, figsize=(8, 6), layout='constrained')
ax1.imshow(m7.T, aspect='auto', norm=LogNorm(), origin='lower', 
           extent=[t_lims[0], t_lims[1], freq_m5[0], freq_m7[-1]])
ax1.tick_params(axis='x', which='both', bottom=False, top=False, labelbottom=False)
ax2.imshow(m5.T, aspect='auto', norm=LogNorm(), origin='lower',
           extent=[t_lims[0], t_lims[1], freq_m5[0], freq_m5[-1]])
ax2.tick_params(axis='x', which='both', bottom=False, top=False, labelbottom=False)
ax3.imshow(m3.T, aspect='auto', norm=LogNorm(), origin='lower',
           extent=[t_lims[0], t_lims[1], freq_m3[0], freq_m3[-1]])
ax3.xaxis_date()
fig.suptitle(f'Mode 357 {obs_start}')